In [1]:
!sudo apt install git-lfs --upgrade

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [2]:
!git init .
!git lfs install

Initialized empty Git repository in /content/.git/
Updated git hooks.
Git LFS initialized.


In [3]:
%%capture
!pip install datasets transformers wandb

In [4]:
import wandb
from datasets import load_dataset
from transformers import (
    AutoFeatureExtractor,
    AutoModelForImageClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.model_selection import StratifiedShuffleSplit
from torchvision.transforms import (
    CenterCrop,
    RandomErasing,
    RandomAutocontrast,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    RandomAdjustSharpness,
    ToTensor,
)
import torch
import datasets
import numpy as np
from datasets import load_metric

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [5]:
%env WANDB_PROJECT=imagein
%env WANDB_ENTITY=davanstrien

env: WANDB_PROJECT=imagein
env: WANDB_ENTITY=davanstrien


In [6]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [35]:
MODEL_CHECKPOINT = "facebook/convnext-base-224"
DATASET_CHECKPOINT = "ImageIN/ImageIn_annotations"

In [36]:
FP_16 = torch.cuda.is_available()

In [37]:
dataset = load_dataset(DATASET_CHECKPOINT, split="train")

In [38]:
dataset.column_names

['image',
 'manifest_url',
 'license',
 'label',
 'attribution',
 'id',
 'choice',
 'annotator',
 'annotation_id',
 'created_at',
 'updated_at',
 'lead_time',
 'loaded_image']

In [39]:
dataset = dataset.remove_columns(
    [name for name in dataset.column_names if name not in {"loaded_image", "choice"}]
)

In [40]:
dataset = dataset.rename_column("choice", "labels")

In [41]:
labels = dataset.unique("labels")

In [42]:
dataset = dataset.cast_column("labels", datasets.ClassLabel(names=labels))

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [43]:
dataset = dataset.train_test_split(0.3, stratify_by_column="labels", seed=42)

In [44]:
train_ds, valid_ds = dataset["train"], dataset["test"]

In [45]:
def prepare_transforms(model_checkpoint, train_ds, valid_ds, test_ds=None):
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
    normalize = Normalize(
        mean=feature_extractor.image_mean, std=feature_extractor.image_std
    )
    _train_transforms = Compose(
        [
            Resize((feature_extractor.size, feature_extractor.size)),
            RandomAdjustSharpness(0.1),
            RandomAutocontrast(),
            ToTensor(),
            normalize,
            RandomErasing(),
        ]
    )

    _val_transforms = Compose(
        [
            Resize((feature_extractor.size, feature_extractor.size)),
            ToTensor(),
            normalize,
        ]
    )

    def train_transforms(examples):
        examples["pixel_values"] = [
            _train_transforms(image.convert("RGB"))
            for image in examples["loaded_image"]
        ]
        return examples

    def val_transforms(examples):
        examples["pixel_values"] = [
            _val_transforms(image.convert("RGB")) for image in examples["loaded_image"]
        ]
        return examples

    train_ds.set_transform(train_transforms)
    valid_ds.set_transform(val_transforms)

    return train_ds, valid_ds

In [46]:
train_ds, valid_ds = prepare_transforms(MODEL_CHECKPOINT, train_ds, valid_ds)

loading configuration file preprocessor_config.json from cache at /root/.cache/huggingface/hub/models--facebook--convnext-base-224/snapshots/eda2970bc74154a2af92300316deecd49f72bea8/preprocessor_config.json
Feature extractor ConvNextFeatureExtractor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ConvNextFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 224
}



In [47]:
train_ds

Dataset({
    features: ['labels', 'loaded_image'],
    num_rows: 1327
})

In [48]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [49]:
id2label = dict(enumerate(train_ds.features["labels"].names))
label2id = {v: k for k, v in id2label.items()}

In [50]:
model = AutoModelForImageClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--convnext-base-224/snapshots/eda2970bc74154a2af92300316deecd49f72bea8/config.json
Model config ConvNextConfig {
  "_name_or_path": "facebook/convnext-base-224",
  "architectures": [
    "ConvNextForImageClassification"
  ],
  "depths": [
    3,
    3,
    27,
    3
  ],
  "drop_path_rate": 0.0,
  "hidden_act": "gelu",
  "hidden_sizes": [
    128,
    256,
    512,
    1024
  ],
  "id2label": {
    "0": "not-illustrated",
    "1": "illustrated"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "illustrated": 1,
    "not-illustrated": 0
  },
  "layer_norm_eps": 1e-12,
  "layer_scale_init_value": 1e-06,
  "model_type": "convnext",
  "num_channels": 3,
  "num_stages": 4,
  "patch_size": 4,
  "torch_dtype": "float32",
  "transformers_version": "4.22.1"
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--facebook--convnext-base-2

In [51]:
model_name = MODEL_CHECKPOINT.split("/")[1]
dataset_name = DATASET_CHECKPOINT.split("/")[1]

In [52]:
args = TrainingArguments(
    "imagein",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    hub_model_id=f"ImageIN/{model_name}_finetuned_on_{dataset_name}",
    overwrite_output_dir=True,
    push_to_hub=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.1,
    disable_tqdm=False,
    fp16=FP_16,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir="logs",
    remove_unused_columns=False,
    save_total_limit=1,
    seed=42,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [53]:
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_CHECKPOINT)

loading configuration file preprocessor_config.json from cache at /root/.cache/huggingface/hub/models--facebook--convnext-base-224/snapshots/eda2970bc74154a2af92300316deecd49f72bea8/preprocessor_config.json
Feature extractor ConvNextFeatureExtractor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ConvNextFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 224
}



In [56]:
def compute_metrics(eval_pred):
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    f1_metric = load_metric("f1")
    accuracy_metric = load_metric("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_metric.compute(
        predictions=predictions, references=labels, average="macro"
    )["precision"]
    recall = recall_metric.compute(
        predictions=predictions, references=labels, average="macro"
    )["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")[
        "f1"
    ]
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)[
        "accuracy"
    ]
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [55]:
from huggingface_hub import notebook_login

In [30]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [31]:
!git config --global credential.helper store

In [57]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
)

/content/imagein is already a clone of https://huggingface.co/ImageIN/convnext-base-224_finetuned_on_ImageIn_annotations. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend


In [58]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1327
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4150
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.136823,0.974817,0.963195,0.968832,0.977153
2,No log,0.073367,0.975038,0.972744,0.973884,0.980668


***** Running Evaluation *****
  Num examples = 569
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


Saving model checkpoint to imagein/checkpoint-83
Configuration saved in imagein/checkpoint-83/config.json
Model weights saved in imagein/checkpoint-83/pytorch_model.bin
Feature extractor saved in imagein/checkpoint-83/preprocessor_config.json
Feature extractor saved in imagein/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 569
  Batch size = 16
Saving model checkpoint to imagein/checkpoint-166
Configuration saved in imagein/checkpoint-166/config.json
Model weights saved in imagein/checkpoint-166/pytorch_model.bin
Feature extractor saved in imagein/checkpoint-166/preprocessor_config.json
Deleting older checkpoint [imagein/checkpoint-83] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 569
  Batch size = 16


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.136823,0.974817,0.963195,0.968832,0.977153
2,No log,0.073367,0.975038,0.972744,0.973884,0.980668
3,No log,0.069250,0.975038,0.972744,0.973884,0.980668
4,No log,0.069837,0.975038,0.972744,0.973884,0.980668
5,No log,0.068841,0.975038,0.972744,0.973884,0.980668
6,No log,0.069015,0.972882,0.975150,0.974009,0.980668
7,0.094700,0.066605,0.968876,0.979962,0.974254,0.980668
8,0.094700,0.064229,0.968876,0.979962,0.974254,0.980668
9,0.094700,0.079012,0.976315,0.976315,0.976315,0.982425
10,0.094700,0.081251,0.975038,0.972744,0.973884,0.980668


Saving model checkpoint to imagein/checkpoint-249
Configuration saved in imagein/checkpoint-249/config.json
Model weights saved in imagein/checkpoint-249/pytorch_model.bin
Feature extractor saved in imagein/checkpoint-249/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 569
  Batch size = 16
Saving model checkpoint to imagein/checkpoint-332
Configuration saved in imagein/checkpoint-332/config.json
Model weights saved in imagein/checkpoint-332/pytorch_model.bin
Feature extractor saved in imagein/checkpoint-332/preprocessor_config.json
Deleting older checkpoint [imagein/checkpoint-249] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 569
  Batch size = 16
Saving model checkpoint to imagein/checkpoint-415
Configuration saved in imagein/checkpoint-415/config.json
Model weights saved in imagein/checkpoint-415/pytorch_model.bin
Feature extractor saved in imagein/checkpoint-415/preprocessor_config.json
Deleting older checkpoint [imagein/chec

TrainOutput(global_step=4150, training_loss=0.02293248847306493, metrics={'train_runtime': 13997.5317, 'train_samples_per_second': 4.74, 'train_steps_per_second': 0.296, 'total_flos': 5.247560434933555e+18, 'train_loss': 0.02293248847306493, 'epoch': 50.0})

In [59]:
trainer.push_to_hub("training finished")

Saving model checkpoint to imagein
Configuration saved in imagein/config.json
Model weights saved in imagein/pytorch_model.bin
Feature extractor saved in imagein/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/334M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/ImageIN/convnext-base-224_finetuned_on_ImageIn_annotations
   0ad80c6..806682f  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/ImageIN/convnext-base-224_finetuned_on_ImageIn_annotations
   0ad80c6..806682f  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Image Classification', 'type': 'image-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.9721650326797385}, {'name': 'Recall', 'type': 'recall', 'value': 0.981127206127206}, {'name': 'F1', 'type': 'f1', 'value': 0.976539565260415}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9824253075571178}]}
To https://huggingface.co/ImageIN/convnext-base-224_finetuned_on_ImageIn_annotations
   806682f..465db3b  main -> main

   806682f..465db3b  main -> main



'https://huggingface.co/ImageIN/convnext-base-224_finetuned_on_ImageIn_annotations/commit/806682f1c4021bf3bf36f4a6c4087c6966d62cb0'

In [60]:
wandb.finish()

eval/accuracy,▁▃▃▃▃▃▃▅▃▅▃▅▂▆▇▅▆▆▃▇▅▆▅▁▆▇▇█▇█▇▇▇▆▇▇▇▇▆▅
eval/f1,▁▃▃▃▃▄▄▅▃▅▃▅▂▆▇▅▆▆▃▇▅▆▅▂▆▇▇█▇█▇▇▇▆▇▇▇▇▆▅
eval/loss,█▂▂▂▂▁▁▂▃▂▃▂▄▂▂▂▃▃▃▂▃▃▂▆▂▂▃▁▂▁▂▂▂▂▂▂▂▂▂▂
eval/precision,▆▆▆▆▆▄▄▇▆▇▆▅▆▇▇▅▇▇▆█▇▇▄▁▅▆▆█▆▇██▆▅▆▆██▆▅
eval/recall,▁▄▄▄▄▅▅▅▄▅▄▆▃▅▇▆▅▅▄▆▅▅▇▆▇██▇██▆▆█▇██▆▆▇▆
eval/runtime,▃█▁▂▂▁▂▂▁▃▁▃▄▂▂▂▂▃▃▂▂▄▃▃▃▃▂▃▃▃▄▂▂▃▃▃▃▂▃▂
eval/samples_per_second,▆▁█▇▇█▇▇█▆█▆▅▇▇▇▆▅▆▇▇▅▆▆▆▆▆▆▆▆▅▇▇▆▆▆▆▇▆▇
eval/steps_per_second,▆▁█▇▇█▇▇█▆█▆▅▇▇▇▆▅▆▇▇▅▆▆▆▆▇▆▆▆▅▇▇▆▆▆▆▇▆▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,█▇▆▅▄▃▂▁
